In [1]:
import findspark
findspark.init('/opt/spark/2.4.4/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [ ]:
'''
input

id|month   |sale|
--|--------|----|
 1|Jan-2019|100 |
 2|Jan-2019|200 |
 3|Feb-2019|500 |
 4|Mar-2019|300 |
 5|Mar-2019|100 |


output

+---+--------+--------------+
| id|   month|cumilative_sum|
+---+--------+--------------+
|  1|Jan-2019|           100|
|  2|Jan-2019|           300|
|  3|Feb-2019|           500|
|  4|Mar-2019|           300|
|  5|Mar-2019|           400|
+---+--------+--------------+
'''

In [4]:
spark = SparkSession.\
        builder.\
        appName('Cumulative_Sum'). \
        enableHiveSupport(). \
        getOrCreate()
sc = spark.sparkContext

In [5]:
data = [(1,'Jan-2019',100),
       (2,'Jan-2019',200),
       (3,'Feb-2019',500),
       (4,'Mar-2019',300),
       (5,'Mar-2019',100)]

In [6]:
input_schema = StructType([
                    StructField('id', IntegerType(), True),
                    StructField('month', StringType(), True),
                    StructField('sale', IntegerType(), True)
                ])

In [7]:
df = spark.createDataFrame(data=data, schema=input_schema)
df.show()

+---+--------+----+
| id|   month|sale|
+---+--------+----+
|  1|Jan-2019| 100|
|  2|Jan-2019| 200|
|  3|Feb-2019| 500|
|  4|Mar-2019| 300|
|  5|Mar-2019| 100|
+---+--------+----+



In [8]:
df.createOrReplaceTempView('sales_month_cumilative')

In [11]:
df_new = spark.sql('''
SELECT id, month, sum(sale) over (partition by month order by id) as cumilative_sum from sales_month_cumilative order by id
''')

In [12]:
df_new.show()

+---+--------+--------------+
| id|   month|cumilative_sum|
+---+--------+--------------+
|  1|Jan-2019|           100|
|  2|Jan-2019|           300|
|  3|Feb-2019|           500|
|  4|Mar-2019|           300|
|  5|Mar-2019|           400|
+---+--------+--------------+

